In [58]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.neural_network import MLPClassifier

In [10]:
# Load our csv as a pandas dataframe
btc = pd.read_csv("../../Python/side-projects/btc-price-tracker-enhanced/BTCUSDT-D.csv")

In [29]:
# Create our targets for our dataset
direction = []
for row in range(len(btc)):
    if btc["close"][row] > btc["open"][row]:
        direction.append("long")
    else:
        direction.append("short")
btc["targets"] = direction

In [31]:
# Function to calculate the difference from one day to the next of a given column of data
def percent_change(dataframe, column_name):

    variable = dataframe[column_name]
    change = [0]
    
    for row in range(len(dataframe)):

        if row == 0:
            continue

        difference = round((variable[row] - variable[row - 1]) / variable[row-1] * 100, 2)

        if difference > 0:
            change.append(1)
        else:
            change.append(0)
        
    return change

In [35]:
btc["daily_change"] = percent_change(btc, "open")
btc.head()

,date,time,open,high,low,close,volume,daily_change,targets
0,Thu-23-Dec-21,00:00:00,48610,51432,48030,50794,2246508966,0,long
1,Fri-24-Dec-21,00:00:00,50794,51980,50402,50785,1837726929,1,short
2,Sat-25-Dec-21,00:00:00,50785,51149,50100,50406,1147589024,0,short
3,Sun-26-Dec-21,00:00:00,50406,51294,49283,50763,1676805639,0,long
4,Mon-27-Dec-21,00:00:00,50763,52119,50464,50706,1765283382,1,short


In [37]:
# encode our targets
lb = LabelEncoder()
targets = lb.fit_transform(btc["targets"])

In [72]:
# Creaate a list of our feautres we want to use and split our dataset

features = btc.filter(["daily_change"])

X_train, X_test, y_train, y_test = train_test_split(features, targets, random_state = 42)

# Convert to an array of the numpy variety
X_train = X_train.values
X_test = X_test.values

In [74]:
# Train a neuronboi
mlp = MLPClassifier(hidden_layer_sizes=(100,100,100,100,100), random_state = 42)

In [78]:
# get the boi score
score = cross_val_score(mlp, X_train, y_train, scoring = "accuracy")
score.mean()

0.49669757174392937